# 

## Import Statements

In [1]:
import pandas as pd
import numpy as np
from astropy.units import Quantity, UnitConversionError
from astropy.cosmology import LambdaCDM, WMAP9
from typing import Union, List
from shutil import rmtree, copyfile
import os
from regions import read_ds9, write_ds9
import matplotlib
from matplotlib import pyplot as plt
from regions import EllipsePixelRegion

# This adds the directory above to the path, allowing me to import the common functions that I've written in
#  common.py - this just saves me repeating boring code and makes sure its all consistent
import sys
sys.path.insert(0, '..')
from common import xcs3p_colour, xcs_cosmo, xxlgc100_colour, xxl_cosmo, locuss_colour, locuss_cosmo

import xga
# This just sets the number of cores this analysis is allowed to use
xga.NUM_CORES = 20
# This is a bodge that will only work because xga_output in notebooks has already been defined, XGA
#  will be made to handle this more gracefully at some point
temp_dir = xga.OUTPUT
actual_dir = temp_dir.split('notebooks/')[0]+'notebooks/xga_output/'
xga.OUTPUT = actual_dir
xga.utils.OUTPUT = actual_dir
# As currently XGA will setup an xga_output directory in our current directory, I remove it to keep it all clean
if os.path.exists('xga_output'):
    rmtree('xga_output')
from xga.samples import ClusterSample
from xga.products import Image
from xga.imagetools.misc import physical_rad_to_pix
from xga import xga_conf

import warnings
warnings.filterwarnings('ignore')

## Defining useful functions

In [2]:
def comp_reg(reg_one, reg_two):
    x_pos = reg_one.center.x == reg_two.center.x
    y_pos = reg_one.center.y == reg_two.center.y

    if type(reg_one) == EllipsePixelRegion:       
        height = reg_one.height == reg_two.height
        width = reg_one.width == reg_two.width
        angle = reg_one.angle == reg_two.angle
#         print(reg_one.height, reg_two.height)
#         print(height, width, angle)
#         print('')
        
        victory = x_pos & y_pos & height & width & angle
        
    else:
        radius = reg_one.radius == reg_two.radius
        victory = x_pos & y_pos & radius
    
    return victory

## SDSSRM-XCS

In [3]:
xcs_pth = '../../outputs/custom_regions/sdssrm-xcs/'
mod_xcs_reg_names = [n for n in os.listdir(xcs_pth) if len([f for f in os.listdir(xcs_pth+n) if '.reg' in f]) != 0]

xcs_need_corr_names = [n for n in mod_xcs_reg_names if len([f for f in os.listdir(xcs_pth+n) if 'ogpix' in f]) == 0]
xcs3p = pd.read_csv("../../sample_files/xcs3p_sdssrm_vol_lim_temperr_25%_clusters.csv")
xcs3p = xcs3p[xcs3p['name'].isin(xcs_need_corr_names)]

In [4]:
if len(xcs3p) != 0:
    # Reading out the relevant values into arrays just for ease of passing into the ClusterSample object
    ra = xcs3p['xapa_ra'].values
    dec = xcs3p['xapa_dec'].values
    z = xcs3p['z'].values
    # Not using the IAU names in XCS_NAME column, its easier for me to use the name based on redMaPPer ID
    n = xcs3p['name'].values
    # In arcminutes, ClusterSample declaration will convert to kpc using the provided cosmology
    r500 = Quantity(xcs3p['r500'].values, 'arcmin')
    # Not likely to use richness in this notebook, but I'm putting it in the sample object anyway
    r = xcs3p['richness'].values
    r_err = xcs3p['richness_err'].values

    # Declaring the actual ClusterSample instance for the XCS sample
    xcs_srcs = ClusterSample(ra, dec, z, n, r500=r500, richness=r, richness_err=r_err, cosmology=xcs_cosmo, 
                             load_fits=True, use_peak=True, clean_obs=True, clean_obs_reg='r500', 
                             clean_obs_threshold=0.7)
else:
    xcs_srcs = []

In [5]:
for src in xcs_srcs:
    crt = src.get_combined_ratemaps(lo_en=Quantity(0.5, 'keV'), hi_en=Quantity(2.0, 'keV'))
    cur_pth = xcs_pth + src.name
    
    reg_files = [f for f in os.listdir(cur_pth) if '.reg' in f and 'sky' not in f and 'ogpix' not in f]
    
    reg_file = [f for f in reg_files if 'new' in f][0]
    ds9_regs = read_ds9(cur_pth+'/'+reg_file)
    sky_regs = [r.to_sky(crt.radec_wcs) for r in ds9_regs]
    new_reg_file = 'sky_'+reg_file
    write_ds9(sky_regs, cur_pth+'/'+new_reg_file)
    
    for reg_file in [r for r in reg_files if 'new' not in r]:
        obs_id = reg_file.split('.')[0].split('_')[-1]
        rt = src.get_ratemaps(obs_id=obs_id)[0]
        ds9_regs = read_ds9(cur_pth+'/'+reg_file)
        # This one is converted using the combined image used for the modification
        sky_regs = [r.to_sky(crt.radec_wcs) for r in ds9_regs]
        
        try:
            saucy = [src.source_back_regions('region', obs_id)[0]]
        except AttributeError:
            saucy = []
            
        sky_regs += saucy

        # Then we go back from sky to pixels but pixels for the individual ObsID
        corr_pix_regs = [r.to_pixel(rt.radec_wcs) for r in sky_regs]

        new_reg_file = 'ogpix_'+reg_file
        write_ds9(corr_pix_regs, cur_pth+'/'+new_reg_file, 'image', radunit='')

In [6]:
for name in mod_xcs_reg_names:
    cur_pth = xcs_pth + name
    reg_file = [f for f in os.listdir(cur_pth) if 'sky' in f][0]
    
    regs = read_ds9(cur_pth+'/'+reg_file)
    if len(regs) != 0:
        copyfile(cur_pth + '/' + reg_file, '../../notebooks/xga_output/regions/{n}/{n}_custom.reg'.format(n=name))

## XXL-100-GC

In [7]:
xxl_pth = '../../outputs/custom_regions/xxl/'
mod_xxl_reg_names = [n for n in os.listdir(xxl_pth) if len([f for f in os.listdir(xxl_pth+n) if '.reg' in f]) != 0]

xxl_need_corr_names = [n for n in mod_xxl_reg_names if len([f for f in os.listdir(xxl_pth+n) if 'ogpix' in f]) == 0]

xxlgc100 = pd.read_csv("../../sample_files/xxl_gc100.csv")

# Limit the comparison to clusters with a flag of 0 - meaning it is in the main sample of 100 clusters
xxlgc100 = xxlgc100[xxlgc100['Flag'] == 0]

# Excluding a specific cluster which was excluded in the XXL analysis
xxlgc100 = xxlgc100[xxlgc100['XLSSC'] != 504]
xxlgc100 = xxlgc100[xxlgc100['name'].isin(xxl_need_corr_names)]

In [8]:
if len(xxlgc100) > 0:
    # Reading out the relevant values into arrays just for ease of passing into the ClusterSample object
    ra = xxlgc100['ra'].values
    dec = xxlgc100['dec'].values
    z = xxlgc100['z'].values
    n = xxlgc100['name'].values
    r500 = Quantity(xxlgc100['r500MT'].values, 'Mpc')

    # Declaring the actual ClusterSample instance for the XXL sample
    # This is the only sample whose original analysis used the WMAP9 cosmology
    xxl_srcs = ClusterSample(ra, dec, z, n, r500=r500, cosmology=xxl_cosmo, load_fits=True, use_peak=False, 
                             clean_obs=True, clean_obs_reg='r500', clean_obs_threshold=0.7)
else:
    xxl_srcs = []

In [9]:
for src in xxl_srcs:
    crt = src.get_combined_ratemaps(lo_en=Quantity(0.5, 'keV'), hi_en=Quantity(2.0, 'keV'))
    cur_pth = xxl_pth + src.name
    
    reg_files = [f for f in os.listdir(cur_pth) if '.reg' in f and 'sky' not in f and 'ogpix' not in f]
    
    reg_file = [f for f in reg_files if 'new' in f][0]
    ds9_regs = read_ds9(cur_pth+'/'+reg_file)
    sky_regs = [r.to_sky(crt.radec_wcs) for r in ds9_regs]
    new_reg_file = 'sky_'+reg_file
    write_ds9(sky_regs, cur_pth+'/'+new_reg_file)
    
    for reg_file in [r for r in reg_files if 'new' not in r]:
        obs_id = reg_file.split('.')[0].split('_')[-1]
        rt = src.get_ratemaps(obs_id=obs_id)[0]
        ds9_regs = read_ds9(cur_pth+'/'+reg_file)
        # This one is converted using the combined image used for the modification
        sky_regs = [r.to_sky(crt.radec_wcs) for r in ds9_regs]
        
        try:
            saucy = [src.source_back_regions('region', obs_id)[0]]
        except AttributeError:
            saucy = []
            
        sky_regs += saucy

        # Then we go back from sky to pixels but pixels for the individual ObsID
        corr_pix_regs = [r.to_pixel(rt.radec_wcs) for r in sky_regs]

        new_reg_file = 'ogpix_'+reg_file
        write_ds9(corr_pix_regs, cur_pth+'/'+new_reg_file, 'image', radunit='')

In [10]:
for name in mod_xxl_reg_names:
    cur_pth = xxl_pth + name
    reg_file = [f for f in os.listdir(cur_pth) if 'sky' in f][0]
    
    regs = read_ds9(cur_pth+'/'+reg_file)
    if len(regs) != 0:
        copyfile(cur_pth + '/' + reg_file, '../../notebooks/xga_output/regions/{n}/{n}_custom.reg'.format(n=name))

## LoCuSS High-$L_{\rm{X}}$

In [11]:
loc_pth = '../../outputs/custom_regions/locuss/'
mod_loc_reg_names = [n for n in os.listdir(loc_pth) if len([f for f in os.listdir(loc_pth+n) if '.reg' in f]) != 0]

loc_need_corr_names = [n for n in mod_loc_reg_names if len([f for f in os.listdir(loc_pth+n) if 'ogpix' in f]) == 0]

xxlgc100 = pd.read_csv("../../sample_files/xxl_gc100.csv")

locuss = pd.read_csv("../../sample_files/locuss_highlx_clusters.csv", dtype={'chandra_id': str, 'xmm_obsid': str})
locuss = locuss[locuss['name'].isin(loc_need_corr_names)]

In [12]:
if len(locuss) > 0:
    # Reading out the relevant values into arrays just for ease of passing into the ClusterSample object
    ra = locuss['ra'].values
    dec = locuss['dec'].values
    z = locuss['z'].values
    n = locuss['name'].values
    r500 = Quantity(locuss['r500'].values, 'kpc')
    r2500 = Quantity(locuss['r2500'].values, 'kpc')


    # Declaring the actual ClusterSample instance for the LoCuSS sample
    locuss_srcs = ClusterSample(ra, dec, z, n, r500=r500, r2500=r2500, cosmology=locuss_cosmo, load_fits=True, 
                                use_peak=True, clean_obs=True, clean_obs_reg='r500', clean_obs_threshold=0.7)
else:
    locuss_srcs = []

In [13]:
for src in locuss_srcs:
    crt = src.get_combined_ratemaps(lo_en=Quantity(0.5, 'keV'), hi_en=Quantity(2.0, 'keV'))
    cur_pth = loc_pth + src.name
    
    reg_files = [f for f in os.listdir(cur_pth) if '.reg' in f and 'sky' not in f and 'ogpix' not in f]
    
    reg_file = [f for f in reg_files if 'new' in f][0]
    ds9_regs = read_ds9(cur_pth+'/'+reg_file)
    sky_regs = [r.to_sky(crt.radec_wcs) for r in ds9_regs]
    new_reg_file = 'sky_'+reg_file
    write_ds9(sky_regs, cur_pth+'/'+new_reg_file)
    
    for reg_file in [r for r in reg_files if 'new' not in r]:
        obs_id = reg_file.split('.reg')[0].split('_')[-1]
        rt = src.get_ratemaps(obs_id=obs_id)[0]
        ds9_regs = read_ds9(cur_pth+'/'+reg_file)
        # This one is converted using the combined image used for the modification
        sky_regs = [r.to_sky(crt.radec_wcs) for r in ds9_regs]
        
        try:
            saucy = [src.source_back_regions('region', obs_id)[0]]
        except AttributeError:
            saucy = []
            
        sky_regs += saucy

        # Then we go back from sky to pixels but pixels for the individual ObsID
        corr_pix_regs = [r.to_pixel(rt.radec_wcs) for r in sky_regs]

        new_reg_file = 'ogpix_'+reg_file
        write_ds9(corr_pix_regs, cur_pth+'/'+new_reg_file, 'image', radunit='')

In [14]:
for name in mod_loc_reg_names:
    cur_pth = loc_pth + name
    reg_file = [f for f in os.listdir(cur_pth) if 'sky' in f][0]
    
    regs = read_ds9(cur_pth+'/'+reg_file)
    if len(regs) != 0:
        copyfile(cur_pth + '/' + reg_file, '../../notebooks/xga_output/regions/{n}/{n}_custom.reg'.format(n=name))

## Parsing and copying modified ObsID region files

In [15]:
new_regs = {}
cust_reg_paths = [xcs_pth, xxl_pth, loc_pth]

for ns_ind, name_set in enumerate([mod_xcs_reg_names, mod_xxl_reg_names, mod_loc_reg_names]):
    for name in name_set:
        cur_pth = cust_reg_paths[ns_ind] + name
        reg_files = [f for f in os.listdir(cur_pth) if 'ogpix' in f]
        for rf in reg_files:
            obs_id = rf.split('.reg')[0].split('_')[-1]
            if obs_id not in new_regs:
                new_regs[obs_id] = [cust_reg_paths[ns_ind] + name + '/' + rf]
            else:
                new_regs[obs_id].append(cust_reg_paths[ns_ind] + name + '/' + rf)

In [18]:
new_reg_path = '../xga_output/{o}/{o}_xga_copy.reg'
for obs_id, reg_files in new_regs.items():
    if len(reg_files) > 1:
        cur_regs = [read_ds9(rf) for rf in reg_files]
        
        final_regs = cur_regs[0]
        for reg_set in cur_regs[1:]:
            for reg in reg_set:
                if not any([comp_reg(reg, r) for r in final_regs]):
                    final_regs.append(reg)
                    
        write_ds9(final_regs, new_reg_path.format(o=obs_id), 'image', '')
        
    else:
        copyfile(reg_files[0], new_reg_path.format(o=obs_id))